In [ ]:
%reload_ext autoreload
%autoreload 2

# Useful high-level external modules.
import numpy as np
import pandas as pd
import plotly.express as px

import sys
sys.path.append("../../src")

import data_cleaning
import load_data



In [ ]:
year = 2020
pudl_out = load_data.initialize_pudl_out(year=year)


In [ ]:
# load the fuel content data and visualize the distribution by fuel
fuel_heat_content = pudl_out.gf_eia923()[
    ["plant_id_eia", "report_date", "energy_source_code", "fuel_mmbtu_per_unit"]
]
fuel_heat_content = fuel_heat_content.replace([np.inf, -np.inf], np.nan)
fuel_heat_content = fuel_heat_content.dropna()
px.box(
    fuel_heat_content.sort_values(by="fuel_mmbtu_per_unit"),
    x="energy_source_code",
    y="fuel_mmbtu_per_unit",
    title=f"Range of heat contents for each fuel reported in {year}",
    width=1000,
    height=600
)


In [ ]:
fuel_heat_content[fuel_heat_content["plant_id_eia"] == 902]


In [ ]:
# load EPA fuel type data
epa_fuel_types = data_cleaning.get_epa_unit_fuel_types(year)
epa_fuel_types[epa_fuel_types["energy_source_code"] == "OTH"]


In [ ]:
# Load EIA-860 to examine whether a plant is retired
gens_860 = pudl_out.gens_eia860()
gens_860[gens_860["plant_id_eia"] == 60670]
